In [7]:
# from huggingface_hub import notebook_login
from transformers import WhisperFeatureExtractor
from transformers import WhisperProcessor
from transformers import WhisperTokenizer
from datasets import load_dataset, DatasetDict, concatenate_datasets
from datasets import Audio
from transformers import WhisperForConditionalGeneration
import evaluate
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer, pipeline
from accelerate import PartialState, prepare_pippy
from accelerate.utils import set_seed

from torch.distributed.pipelining import Pipe

from dataclasses import dataclass
from typing import Any, Dict, List, Union

# notebook_login()

In [8]:
model_name = "openai/whisper-small"
cache_dir = '/mnt/8THDD0/storage/huggingface'
set_seed(42)
target_lang = 'zh'
# model_name = "openai/whisper-large-v3-turbo"
output_dir = "/mnt/8THDD0/storage/train/output/"+ model_name.split('/')[1] +"-" + target_lang

In [9]:
tokenizer = WhisperTokenizer.from_pretrained(model_name, language="zh", task="transcribe", cache_dir=cache_dir)

In [16]:
pipe = pipeline(task="automatic-speech-recognition", model=output_dir+'/checkpoint-5000', tokenizer=tokenizer)
try:
    pipe.save_pretrained(output_dir + '/final')
except:
    pipe.save_pretrained(output_dir + '/result')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [11]:
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load

# torch.setnum_threads(1)
common_voice = DatasetDict()

common_voice["test"] = load_dataset("mozilla-foundation/common_voice_14_0", "yue", split="validation", trust_remote_code=True)

common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

processor = WhisperProcessor.from_pretrained("/mnt/8THDD0/storage/train/output/whisper-small-zh/final/")
model = WhisperForConditionalGeneration.from_pretrained("/mnt/8THDD0/storage/train/output/whisper-small-zh/final/").to("cuda")

def map_to_pred(batch):
    audio = batch["audio"]
    input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
    batch["reference"] = processor.tokenizer._normalize(batch['sentence'])

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"))[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch

result = common_voice.map(map_to_pred)


Map:   0%|          | 0/2537 [00:00<?, ? examples/s]

/home/jacky/anaconda3/envs/whisper/lib/python3.12/site-packages/transformers/models/whisper/tokenization_whisper.py:501: UserWarning: The private method `_normalize` is deprecated and will be removed in v5 of Transformers.You can normalize an input string using the Whisper English normalizer using the `normalize` method.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [12]:
cer = load("cer")
# result['test']['reference']
# result['test']["prediction"]
# for i in range(0, len(result['test']['reference'])):
#     print(result['test']['reference'][i])
#     print(result['test']["prediction"][i])
    # print(result['test']['reference'][i] == result['test']["prediction"][i])
print(100 * cer.compute(references=result['test']['reference'], predictions=result['test']["prediction"]))

我已經答咗你啦
我已經答咗你啦
鬼叫你睇明人哋講乜嘢
鬼叫你睇明人哋講乜嘢
你哋水廠做乜撚嘢嘅啫
你哋水廠做乜撚嘢嘅啫
不過我冇同佢傾過囉
不過我冇同佢傾過囉
手機先點擊同步 然後電腦點擊同步
手機先點擊同步 然後電腦點擊同步
唔係想打沉你哋
唔係想打沉你哋
唔係加一行就得喇咩
唔係加一行就得喇咩
唔使擔心
唔使擔心
不過標準鍵盤打好麻煩
不過標準鍵盤打好麻煩
佢唔係你女朋友 我先唔信
佢唔係你女朋友 我先唔信
好似唔多通
好似唔多通
十組先二萬
十組先二萬
所謂處處為民眾設想 都係一句空話
所謂處處為民眾設想 都係一句空話
大家唔堅持 就淪陷得比較快囉
大家唔堅持 就淪陷得比較快囉
有機構開班教拼音
有機構開班教拼音
雖然我硬係有啲唔聚財
雖然我硬係有啲唔聚財
幾好 有中英文對照
幾好 有中英文對照
早晨 今日會有咩傾呢
早晨 今日會有咩傾呢
明顯中咗招都驗唔到出嚟
明顯中咗招都驗唔到出嚟
我度落緊雨
我度落緊雨
哦 你有打返過來
哦 你有打返過來
我唔否認呢個係技術考慮
我唔否認呢個係技術考慮
老鼠尾生瘡 大極有限
老鼠尾生瘡 大極有限
只許州官放火 不准百姓點燈
只許州官放火 不准百姓點燈
國家 好壞並存 唔會十全十美
國家 好壞並存 唔會十全十美
泰語始終用自己文字
泰語始終用自己文字
嗰個網站冇語音嘅
嗰個網站冇語音嘅
你冇病就安靜啦
你冇病就安靜啦
手機響咗下就停 唔通係收到鬼嘅來電
手機響咗下就停 唔通係收到鬼嘅來電
但我都唔知係邊個字
但我都唔知係邊個字
佢哋都好失望
佢哋都好失望
有時提一提我 你知道有乜嘢口音嘅 我順便帶埋入去
有時提一提我 你知道有乜嘢口音嘅 我順便帶埋入去
不如大家出街做返自己
不如大家出街做返自己
當年經典 家陣聽返都仲咁正
當年經典 家陣聽返都仲咁正
表示我都聽唔明
表示我都聽唔明
噉都未聽過
噉都未聽過
有冇得遠程
有冇得遠程
鼠標可唔可以實現語音識別
鼠標可唔可以實現語音識別
去彎彎嗰度睇新聞咯
去彎彎嗰度睇新聞咯
戶外活動直播 各種各樣
戶外活動直播 各種各樣
冇今日嗰個咁搞笑
冇今日嗰個咁搞笑
有時我唔想用咁多時間喺呢啲地方
有時我唔想用咁多時間喺呢啲地方
要畀心機喎
要畀心機喎
睇咗下佢嘅解析 真係離晒譜
睇咗下佢嘅解析 真係離晒譜
全世界 年輕一代同老一輩都係雞同鴨講㗎啦
全世界 年輕一代同老一輩都係雞同鴨講㗎

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import pipeline
from huggingface_hub import notebook_login
import os

# Define the path to your local model directory
model_path = '/mnt/8THDD0/storage/train/output/whisper-small-zh/final/'

# Define the name of your repository on Hugging Face (must include username)
repo_name = 'JackyHoCL/whisper-small-cantonese-yue-english'

# Push the model to the repository
pipeline("automatic-speech-recognition", model=model_path).push_to_hub(repo_name)

print(f"Model successfully uploaded to {repo_name}")


In [ ]:
!ct2-transformers-converter --model JackyHoCL/whisper-small-cantonese-yue-english --output_dir whisper-small-cantonese-yue-english-ct2 --copy_files tokenizer.json preprocessor_config.json --quantization float16

In [ ]:
# #custom evaluation with user defined audio

# from transformers import WhisperProcessor, WhisperForConditionalGeneration
# from datasets import load_dataset
# import torchaudio

# # load model and processor
# # processor = WhisperProcessor.from_pretrained("openai/whisper-small")
# # model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
# # model.config.forced_decoder_ids = None

# audio = torchaudio.load("test1.mp3")
# audio
# # load dummy dataset and read audio files
# # ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
# # sample = ds[0]["audio"]
# # print(sample)
# # input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 

# # # generate token ids
# # predicted_ids = model.generate(input_features)
# # # decode token ids to text
# # transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

# # transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)


(tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -2.6838e-05,
           4.1921e-05,  6.7607e-05],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -3.0045e-05,
           3.9640e-05,  6.8014e-05]]),
 44100)

In [17]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = '/mnt/8THDD0/storage/train/output/whisper-small-zh/final/'

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)


generate_kwargs = {
    "return_timestamps": True
}

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]

result = pipe("test2.mp3", generate_kwargs=generate_kwargs)
print(result["text"])


/home/jacky/anaconda3/envs/whisper/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


阿利生，同埋大部分香港人都知道你係首富啦我唔承認喎係啦，有啲人會排名啦嘛。但係近月就有啲啊，首富地位咁樣，係咪要變動咁樣嘅講法?你對財富或者首富呢樣嘢嘅睇法有咩可以同我問下嗎?我可以同你講下,第一,我唔重視這個首富嘅物業。我用咩做例子呢? 差唔多最單獨有十年。外國嘅雜誌係度話我有幾多財產呢?最少有十年講少咗我一部。為咩呢？佢將嗰個Husky都唔係我嘅，其實Husky我係最大嘅股東。仲大個和基些小個度係私人嘅，但係我又冇出聲，佢最好寫我小碟。你寫我大啲有咩用啫？第一，第二呢，我基金回人，百運之一百係我自己私人嘅詞語。嗰度佔咗我隨產嘅新穎一,捐到已經捐咗出去一百四十五億,已經錢出咗去。你正義度梗係演演多過好多倍去一百四十五億,佢哋問我呢啲錢已經屬於大國,你唔好計我呢條數,咁係另外,你唔會等等淨係和記上市公司有啲苦行嘅,你一定要投資淨係睇你新科技嗰啲,點解你又唔講啲呢?我話嗌嗌啲唔計啦,你淨係計我從來係得啦,個個問題你諗十年我話最可怕佢唔發覺, 唔係到有一日佢發覺咗。估計最單獨佢寫嘅嘢最單獨憎有冇佢差成四十個比選最少?咁個最嚴重嘅第一我呢認為人係咪守護其次,係咪你嘅做人呢係咪做得有意義。人家算你,認為你個人算得過。你嘅人生呢係值得,人家陰陪唔會太疲你,嗰個先係近。關於你什麼時候打算退休的問題？退休太久啦 退休太少 退休個時光你沒有什麼害怕 其實我自己來講 如果我今天太休啦 我還有很多事都做不完的事 我基金會啦 我基金會很多縣府都在住 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都做不完的事 啊 每個月都會有很多事都擔都是對我們自己人有好處的就醫醫療 還有其他很多很多在住當時今天啦 這個世界的投資房間比較複雜一點 複雜所以啦 現在